In [1]:
import time
import pymongo
import os
import numpy as np
import pandas as pd
import requests
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')

# How many documents do we have to scrape and download?

In [14]:
""" A util for making a connection to mongo """
def _connect_mongo(host, port, username, password, db):
    try:
        if username and password:
            mongo_uri = 'mongodb://%s:%s@%s:%s/%s' % (username, password, host, port, db)
            conn = pymongo.MongoClient(mongo_uri)
            
        else:
            conn = pymongo.MongoClient(host, port)

        print "Connected successfully!!!"
        return conn[db]
    
    except pymongo.errors.ConnectionFailure, e:
        print "Could not connect to MongoDB: %s" % e 

def insert_folio(db,collection,insert_query={},
                 host='localhost',port=27017,username=None, password=None,):
    # Connect to MongoDB
    db = _connect_mongo(host=host, port=port, username=username, password=password, db=db)
    return db
    
""" send a query and reutnr the count """   
def mongo_count(db,collection, query={}, host='localhost', port=27017, 
               username=None, password=None, no_id=True):
    # Connect to MongoDB
    db = _connect_mongo(host=host, port=port, username=username, password=password, db=db)
    
    # With query get the count
    count = db[collection].find(query).count()
    return count

""" Read from Mongo and Store into Pandas DataFrame """
def read_mongo(db, collection, query={}, nb_limit=None, host='localhost', port=27017,
               username=None, password=None, no_id=True):

    # Connect to MongoDB
    db = _connect_mongo(host=host, port=port, username=username, password=password, db=db)
    
    if nb_limit == None:
        # just query
        cursor = db[collection].find(query)
    else:
        cursor = db[collection].find(query).limit(nb_limit)
    
    df = pd.DataFrame(list(cursor))
    if no_id :
        del df['_id']
    return df


def _download_file(url):
    local_filename = url.split('/')[-1]
    # NOTE the stream=True parameter
    r = requests.get(url, stream=True, verify = False)
    with open('./'+local_filename, 'wb') as f:
        for chunk in r.iter_content(chunk_size=1024): 
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)
    return local_filename


# we need to change the infilename into something else for the respuestas
def download_file(url,filename,path,extension=""):
    infilename = _download_file(url)
    newname = os.path.splitext(path+filename)[0]+extension
    # yes
    output = os.rename(infilename, newname)
    print "File is located at:",newname
    
    
"""Downloads a stack of urls"""
def download_stack_urls(stackOfUrls,fileNameBase,path):
    count = 0
    length = len(stackOfUrls)
    for url in stackOfUrls:
        r = requests.get(url, verify = False)
        if r.status_code == 200:
            download_file_raw(url,str(fileNameBase)+str(count),path)
        else:
            print r.status_code,"for html url"
            print url
        count+=1
        print "Downloaded",count,"out of",length
        time.sleep(1)
        
def request_to_db(request,db):
    pass

def update_request(request,db):
    pass

def uploadRequest(db):
    pass


In [4]:
df = read_mongo('polisci_test26Oct16',
                'folios',
                {'ARCHIVORESPUESTA' : {'$regex' : 'respuesta=' }},
                400000
               )

Connected successfully!!!


In [5]:
df.head()

,ARCHIVOADJUNTOSOLICITUD,ARCHIVORESPUESTA,CODIGOPOSTAL,DEPENDENCIA,DESCRIPCIONSOLICITUD,ESTADO,ESTATUS,FECHARESPUESTA,FECHASOLICITUD,FOLIO,...,OTROSDATOS,PAIS,PDFOCR,RESPUESTA,SECTOR,TIPOSOLICITUD,attach_full,attach_inc,word_count,year
0,NaN,https://www.infomex.org.mx/gobiernofederal/mod...,4510,SECRETARÍA DE GOBERNACIÓN,Tirajes y circulación de los diarios y revista...,Distrito Federal,Terminada,08/08/2003,12/06/2003 10:42,'0000400000803,...,NaN,México,0,Entrega de información en medio electrónico,Gobernación,Información Pública,,0,18.0,2003
1,NaN,https://www.infomex.org.mx/gobiernofederal/mod...,4800,SECRETARÍA DE GOBERNACIÓN,Información detallada de lo que pagó Secretarí...,Distrito Federal,Terminada,25/07/2003,12/06/2003 11:50,'0000400002003,...,NaN,México,0,Entrega de información en medio electrónico,Gobernación,Información Pública,,0,32.0,2003
2,https://www.infomex.org.mx/gobiernofederal/mod...,https://www.infomex.org.mx/gobiernofederal/mod...,6400,SECRETARÍA DE GOBERNACIÓN,Secretarìa de Gobernacion (VER ARCHIVO DE WORD...,Distrito Federal,Terminada,25/07/2003,12/06/2003 14:43,'0000400004803,...,NaN,México,0,Entrega de información en medio electrónico,Gobernación,Información Pública,Not Found The requested URL /jspsi/document...,1,27.0,2003
3,https://www.infomex.org.mx/gobiernofederal/mod...,https://www.infomex.org.mx/gobiernofederal/mod...,4040,SECRETARÍA DE GOBERNACIÓN,Anexo carta con un punto en el que explicola i...,Distrito Federal,Terminada,24/07/2003,12/06/2003 14:57,'0000400005203,...,Todos estos documentos estan en el archivo de ...,México,0,Entrega de información en medio electrónico,Gobernación,Información Pública,México DF a 12 de junio de 2003 EFRE...,1,195.0,2003
4,https://www.infomex.org.mx/gobiernofederal/mod...,https://www.infomex.org.mx/gobiernofederal/mod...,6400,SECRETARÍA DE GOBERNACIÓN,DIRECCIÓN GENERAL DE RADIO TELEVISIÓN Y CINEM...,Distrito Federal,Terminada,25/07/2003,12/06/2003 15:20,'0000400005403,...,NaN,México,0,Entrega de información en medio electrónico,Gobernación,Información Pública,Dirección General de Radio Televisión y Ci...,1,201.0,2003


In [6]:
np.unique(df['ESTATUS'])

array([u'Desechada por falta de pago',
       u'Desechada por falta de respuesta del ciudadano', u'Terminada'], dtype=object)

In [7]:
df = df.set_index(np.arange(0,len(df)))

In [8]:
print df[['ARCHIVOADJUNTOSOLICITUD']].iloc[3].item()
print df[['ARCHIVORESPUESTA']].iloc[3].item()

https://www.infomex.org.mx/gobiernofederal/moduloPublico/MimeGenerator.action?folio=0000400005203
https://www.infomex.org.mx/gobiernofederal/moduloPublico/MimeAdjuntoSeguimientoGenerator.action?folio=0000400005203&respuesta=6


In [15]:
download_file('https://www.infomex.org.mx/gobiernofederal/moduloPublico/MimeGenerator.action?folio=0000400005203','wordDoc','./','.zip')

File is located at: ./wordDoc.zip


In [16]:
download_file('https://www.infomex.org.mx/gobiernofederal/moduloPublico/MimeGenerator.action?folio=0000400005203','wordDoc','./')

File is located at: ./wordDoc


In [18]:
download_file('https://www.infomex.org.mx/gobiernofederal/moduloPublico/MimeAdjuntoSeguimientoGenerator.action?folio=0000400005203&respuesta=6','azipfile','./')

File is located at: ./azipfile


In [19]:
respuestas = df['ARCHIVORESPUESTA']
ajuntos = df['ARCHIVOADJUNTOSOLICITUD']
# download only 10 files
respuestas = respuestas[0:10]
# download only 10 files
ajuntos = ajuntos[0:10]

#download_stack_urls(respuestas,'arespuesta','./attachments/respuesta/')

In [20]:
ajuntos_clean = []
for url in ajuntos:
    if str(url) == 'nan':
        continue
    else:
        ajuntos_clean.append(url)
#download_stack_urls(ajuntos_clean,'aajunto','./attachments/ajunto/')

In [21]:
df2 = df[df['attach_inc'] == 1]
max(df2['word_count'])

501358.0

In [22]:
df2 = read_mongo('polisci_test26Oct16','folios',{'attach_inc' : 0},400000)

Connected successfully!!!


In [23]:
len(df2[df2['attach_full']== '    '])

400000

In [28]:
not_attachments = mongo_count('polisci_test26Oct16','folios',{'attach_inc' : 0})
total = mongo_count('polisci_test26Oct16','folios')
print "difference",total-not_attachments
print "percent of documents with attachments included",float(not_attachments)/float(total),'%'

Connected successfully!!!
Connected successfully!!!
difference 128692
percent of documents with attachments included 0.898840086655 %
